In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# coding: utf-8
from classifier import Classifier
from neo_models import Neo4j
from random import shuffle
from time import time

In [3]:
# 读取neo4j内容
db = Neo4j()
db.connectDB()
data_set = db.getLabeledHudongItem('labels.txt')

create neo4j class ...
load LabeledHudongItem over ...
got items: 2176


In [4]:
# 读取fasttext的模型
shuffle(data_set)
length = int(len(data_set)/10)

vis = []  # 记录下每个label出现次数，控制每个label出现的个数
for i in range(2):
    vis.append(0)

cur = []  # 让每类样本尽量分布均匀
for p in data_set:
    label = p.label
    if vis[int(label)] > 300 and int(label) != 0:
        continue
    if vis[int(label)] > 300 and int(label) == 0:
        continue
    vis[int(label)] += 1
    cur.append(p)
    print(p.title,p.label)
    print(vis)
    
data_set = cur
print(len(data_set))
# split = len(data_set)*199/200
# split = int(split) # 切分点
# print(split)
# train_set = data_set[:split]
# test_set = data_set[split:]


耳鼻咽喉科 0
[1, 0]
Samuel 0
[2, 0]
废弃物 0
[3, 0]
安徽科技学院 0
[4, 0]
精氨酸 0
[5, 0]
何润东 0
[6, 0]
关亚军 0
[7, 0]
中国联通学院 0
[8, 0]
河北大学外国语学院 0
[9, 0]
太平经 0
[10, 0]
绵阳 0
[11, 0]
引力坍缩 0
[12, 0]
SOH 1
[12, 1]
红土 0
[13, 1]
查理曼帝国 0
[14, 1]
苗述 0
[15, 1]
阿克塞拉·瑞德 0
[16, 1]
沙鱼涌 0
[17, 1]
红钻 0
[18, 1]
qq花园 0
[19, 1]
LTSP 1
[19, 2]
冉晨 0
[20, 2]
路由选择协议 1
[20, 3]
瓦隆-布拉班特省 0
[21, 3]
相声 0
[22, 3]
中央商务平台 0
[23, 3]
Never 0
[24, 3]
弗里堡大学 0
[25, 3]
中外合资 0
[26, 3]
热依娜 0
[27, 3]
异想天开 0
[28, 3]
洪友士 0
[29, 3]
系统软件 1
[29, 4]
IM 0
[30, 4]
威廉·燕卜逊 0
[31, 4]
赵明 0
[32, 4]
Texas 0
[33, 4]
system 1
[33, 5]
冯至 0
[34, 5]
深圳东站 0
[35, 5]
视频会议系统 0
[36, 5]
产生 0
[37, 5]
网络摄像机 1
[37, 6]
安全认证 1
[37, 7]
黑龙会 0
[38, 7]
两税法 0
[39, 7]
直取 0
[40, 7]
闻一多全集 0
[41, 7]
荣耀 1
[41, 8]
硬盘分区 1
[41, 9]
空肠 0
[42, 9]
核弹头 0
[43, 9]
nat 1
[43, 10]
再见陌生人 0
[44, 10]
赫尔辛基 0
[45, 10]
变化 0
[46, 10]
sensation 0
[47, 10]
香港特别行政区政府 0
[48, 10]
加壳 1
[48, 11]
光粒子 0
[49, 11]
幂级数 0
[50, 11]
中搜 1
[50, 12]
Amaze 0
[51, 12]
类脂 0
[52, 12]
太阳大气 0
[53, 12]
中南大学 0
[54, 12]
深圳高新技术产

In [5]:
classifier = Classifier('wiki.zh.bin')

classifier load over...


In [9]:
for p in data_set:
    if p.title=='即时通讯':
        print(p.title,p.label,p.detail)

即时通讯 1 None


In [6]:
def cross_verification(k_cv,data_set,W,k_num):  # k折叠交叉验证
    length = int(len(data_set)/k_cv)
    l = 0
    aver = 0.0
    for i in range(k_cv):
        train_set = data_set[:length*i]
        test_set = data_set[length*i:length*(i+1)]
        train_set.extend(data_set[length*(i+1):])
        classifier.load_trainSet(train_set)
        classifier.set_parameter(weight=W,k=k_num)
        count = 0
        for p in test_set:
            predict = classifier.KNN_predict(p)
            if predict==int(p.label):
                count += 1
            #print(str(p.title)+" "+str(predict)+"  "+str(p.label))
        precision = 1.0*count/len(test_set)
        aver += precision
       #  print("precision: "+str(1.0*count/len(test_set)*100)+"%")
    aver /= k_cv
    # print("aver precision:"+str(aver*100)+"%")
    return aver
    

In [7]:
import logging
logging.basicConfig(level=logging.DEBUG,
                format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                datefmt='%a, %d %b %Y %H:%M:%S',
                filename='mylog.log',
                filemode='w')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

grid_list = [0.1,0.3,1.,3.,10]
W = [0.,0.]
ans = 0.
ans_W = [0.,0.]
count = 0
k = 10  # knn的k

for w2 in grid_list:
    W = [0.3,w2]
    cur = cross_verification(10,data_set,W,k)
    if cur > ans:
        ans = cur
        ans_W = W
        ans_k = k
    count += 1
    logging.info("count:"+str(count)+" precision: "+str(cur)+" now answer: "+" w="+str(ans_W))
logging.info("finished.")

root        : INFO     count:1 precision: 0.8 now answer:  w=[0.3, 0.1]
root        : INFO     count:2 precision: 0.82 now answer:  w=[0.3, 0.3]
root        : INFO     count:3 precision: 0.8233333333333333 now answer:  w=[0.3, 1.0]
root        : INFO     count:4 precision: 0.7983333333333332 now answer:  w=[0.3, 1.0]
root        : INFO     count:5 precision: 0.8016666666666665 now answer:  w=[0.3, 1.0]
root        : INFO     finished.


In [8]:
cur = cross_verification(10,data_set,[0.3,1],10)
ans = -7
ans_W = None
ans_k = None
if cur > ans:
    ans = cur
    ans_W = W
    ans_k = k
logging.info("count:"+str(count)+"precision: "+str(cur)+" now answer: "+" w="+str(ans_W))



root        : INFO     count:5precision: 0.8233333333333333 now answer:  w=[0.3, 10]


In [9]:
from hudong_class import HudongItem
p1 = HudongItem(db.matchHudongItembyTitle('数字信号'))
p2 = HudongItem(db.matchHudongItembyTitle('用户界面'))
classifier.set_parameter(weight=[0.2,0.2,0.2,0.2,0.0],k=10)
classifier.KNN_predict(p1)

1